In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from fastcore.all import *
from fastai.vision.all import *
import torchvision
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder
import time
from sklearn.metrics import precision_score, recall_score, f1_score
import os
from datetime import datetime
import numpy as np
import logging
import sys
import re
from sendMsg import sendMessageToHeathcliff05botToAChat
from makeGraphPlotsAndSendMsg import makeGraphAndSendPhoto

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, input_planes, output_planes,stride,identityFlag):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_planes, output_planes, kernel_size=(3, 3), stride=(stride, stride), padding=(1, 1), bias=False)
        self.bn1 =  nn.BatchNorm2d(output_planes, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(output_planes, output_planes, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.bn2 =  nn.BatchNorm2d(output_planes, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        self.downsample = None
        if identityFlag:
            self.downsample = nn.Sequential(
                nn.Conv2d(input_planes, output_planes, kernel_size=(1, 1), stride=(stride, stride), bias=False),
                nn.BatchNorm2d(output_planes, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            identity = self.downsample(x)
        out = out + identity
        out = self.relu(out)
        return out


class ResNet18(nn.Sequential):
    def __init__(self):
        super(ResNet18,self).__init__()
        self.layer1 = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
                nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
                nn.Sequential(
                    BasicBlock(input_planes=64, output_planes=64,stride=1,identityFlag=False),
                    BasicBlock(input_planes=64, output_planes=64,stride=1,identityFlag=False)
                ),
                nn.Sequential(
                    BasicBlock(input_planes=64, output_planes=128, stride=2, identityFlag=True),
                    BasicBlock(input_planes=128, output_planes=128, stride=1, identityFlag=False)
                ),
                nn.Sequential(
                    BasicBlock(input_planes=128, output_planes=256, stride=2, identityFlag=True),
                    BasicBlock(input_planes=256, output_planes=256, stride=1, identityFlag=False)
                ),
                nn.Sequential(
                    BasicBlock(input_planes=256, output_planes=512, stride=2, identityFlag=True),
                    BasicBlock(input_planes=512, output_planes=512, stride=1, identityFlag=False)
                )
            )

        self.layer2 = nn.Sequential(
                AdaptiveConcatPool2d(1),
                Flatten(full=False),
                nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Dropout(p=0.25, inplace=False),
                nn.Linear(in_features=1024, out_features=512, bias=False),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Dropout(p=0.5, inplace=False),
                nn.Linear(in_features=512, out_features=6, bias=False)
            )
        def forward(self,x):
            out = self.layer1(x)
            out = self.layer2(out)
            return out

In [3]:
pstartTime = datetime.now()
formatted_date_pstartTime = pstartTime.strftime('%Y-%m-%d %H:%M:%S')

batch_size = 512
lrn_rate = 0.001
num_epochs = 50
seed_value = 60
wt_decay = 0
file_path_to_save_stats = "../stats/statsFullTrainResnetStrikesBack.csv"
file_path_to_save_model = "../model/mlprojectFullTrainResnetStrikesBack.pth"
dataset_root = '../data/garmentStructuredData'
log_file_path = "../logs/"+formatted_date_pstartTime+".log"
dir_to_save_plots = "../plots/"+formatted_date_pstartTime+".png"

In [4]:
model = ResNet18()

In [5]:
import safetensors

In [26]:
tensors = {}
with safetensors.safe_open("C:/Users/sriha/NEU/ML Project/models/imagenet_resnet18.safetensors", framework="pt") as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

In [27]:
import json
with open('C:/Users/sriha/NEU/ML Project/awsMLProject/finetune_model_param_map.json', 'r') as file:
    mapDoc = json.load(file)

In [39]:
for name, param in model.named_parameters():
    if name in mapDoc.keys():
        param.data.copy_(tensors[mapDoc[name]])
    else :
        print(name,"param not copied")


layer2.2.weight param not copied
layer2.2.bias param not copied
layer2.4.weight param not copied
layer2.6.weight param not copied
layer2.6.bias param not copied
layer2.8.weight param not copied


In [38]:
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if re.search("layer2",name):
        print(name,"setting requires_grad True")
        param.requires_grad = True


layer2.2.weight setting requires_grad True
layer2.2.bias setting requires_grad True
layer2.4.weight setting requires_grad True
layer2.6.weight setting requires_grad True
layer2.6.bias setting requires_grad True
layer2.8.weight setting requires_grad True
